# Tester

In [1]:
import PIL
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from matplotlib import pyplot as plt
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
from torchvision.models import resnet50
import math
from tqdm import tqdm_notebook as tqdm

from puzzle_vit import PuzzleViT
from puzzle_res50 import PuzzleCNNCoord
from util.tester import visualDoubleLoss


gpu_ids = []
device_names = []
if torch.cuda.is_available():
    for gpu_id in range(torch.cuda.device_count()):
        gpu_ids += [gpu_id]
        device_names += [torch.cuda.get_device_name(gpu_id)]
print(gpu_ids)
print(device_names)

if len(gpu_ids) > 1:
    gpu = 'cuda:' + str(gpu_ids[3])  # GPU Number
else:
    gpu = "cuda" if torch.cuda.is_available() else "cpu"

Files already downloaded and verified
Files already downloaded and verified
[0, 1, 2, 3]
['NVIDIA GeForce RTX 3090', 'NVIDIA GeForce RTX 3090', 'NVIDIA GeForce RTX 3090', 'NVIDIA GeForce RTX 3090']


## Hyper parameter

In [2]:
device = gpu
LEARNING_RATE = 1e-05
BATCH_SIZE = 64
NUM_EPOCHS = 100
NUM_WORKERS = 2
test_model_path = './save/puzzle_imagenet_1000_vit_ep100_lr1e-05_b64_c.pt'

## Dataset

In [3]:
transform = transforms.Compose([
    transforms.Resize(256, interpolation=PIL.Image.BICUBIC),
    transforms.CenterCrop(224),
    transforms.Pad(padding=(0, 0, 1, 1)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder('../datasets/ImageNet/train', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
val_dataset = Subset(train_dataset, list(range(int(0.01*len(train_dataset)))))
val_loader = DataLoader(dataset=val_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
test_dataset = datasets.ImageFolder('../datasets/ImageNet/val', transform=transform)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)

## Tester class

In [4]:
class Tester(object):
    def __init__(self):
        self.model = None
        self.optimizer = None
        self.epochs = [0]
        self.losses_c = [0]
        self.losses_t = [0]
        self.accuracies = [0]

    def process(self, load=True):
        self.build_model(load)
        self.eval_model()
    
    def build_model(self, load):
        self.model = PuzzleViT().to(device)
        print(f'Parameter: {sum(p.numel() for p in self.model.parameters() if p.requires_grad)}')
        if load:
            checkpoint = torch.load(test_model_path)
            self.epochs = checkpoint['epochs']
            self.model.load_state_dict(checkpoint['model'])
            self.losses_c = checkpoint['losses_coord']
            self.losses_t = checkpoint['losses_total']
#             self.accuracies = checkpoint['accuracies']
            print(f'Parameter: {sum(p.numel() for p in self.model.parameters() if p.requires_grad)}')
            print(f'Epoch: {self.epochs[-1]}')

    def eval_model(self, epoch=-1):
        model = self.model

        model.eval()

        total = 0
        diff = 0
        correct = 0
        with torch.no_grad():
            for batch_idx, (inputs, _) in tqdm(enumerate(test_loader, 0), total=len(test_loader)):
                inputs = inputs.to(device)

                outputs, labels, _ = model(inputs)

                pred = outputs
                total += labels.size(0)
                diff += (torch.dist(pred, labels)).sum().item()
                pred_ = model.mapping(pred)
                labels_ = model.mapping(labels)
                correct += (pred_ == labels_).all(dim=2).sum().item()

        print(f'[Epoch {epoch + 1}] Avg diff on the test set: {diff / total:.2f}')
        print(f'[Epoch {epoch + 1}] Accuracy on the test set: {100 * correct / (total * labels.size(1)):.2f}%')
        torch.set_printoptions(precision=2)
        total = labels.size(1)
        correct = (pred_[0] == labels_[0]).all(dim=1).sum().item()
        print(f'[Sample result]')
        print(torch.cat((pred_[0], labels_[0]), dim=1))
        print(f'Accuracy: {100 * correct / total:.2f}%')
        
    def loss_checker(self):
        checkpoint = torch.load(test_model_path, map_location='cpu')
        self.epochs = checkpoint['epochs']
        self.losses_c = checkpoint['losses_coord']
        self.losses_t = checkpoint['losses_total']
        print(f'Steps: {len(self.epochs)} (x100) steps')
        ls_x = []
        ls_y = []
        for x, y in enumerate(self.losses_t):
            ls_x.append(x)
            ls_y.append(y)
        plt.plot(ls_x, ls_y)
        plt.title('Losses_t Plot')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.show()
        [print(f'Average Loss: {i:.3f}') for i in self.losses_t]
    
    def lr_checker(self):
        self.build_model(load=True)
        model = self.model
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=0.9)
        scheduler = CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS)
        
        ls_epoch = []
        ls_lr = []
        for epoch in range(NUM_EPOCHS):
            running_loss = 0.0
            saving_loss = 0.0
            ls_epoch.append(epoch)
            ls_lr.append(optimizer.param_groups[0]["lr"])
            scheduler.step()
        plt.plot(ls_epoch, ls_lr)
        plt.title('LR Plot')
        plt.xlabel('Epoch')
        plt.ylabel('LR')
        plt.show()
        
    def lr_compare(self):
        LEARNING_RATE = 1e-05
        NUM_EPOCHS = 100
        self.build_model(load=True)
        model = self.model
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=0.9)
        scheduler = CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS)
        ls_epoch = []
        ls_lr = []
        for epoch in range(NUM_EPOCHS):
            running_loss = 0.0
            saving_loss = 0.0
            ls_epoch.append(epoch)
            ls_lr.append(optimizer.param_groups[0]["lr"])
            scheduler.step()
        plt.plot(ls_epoch, ls_lr, label='Curve 1')
        
        LEARNING_RATE = 2e-05
        NUM_EPOCHS = 50
        model = self.model
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=0.9)
        scheduler = CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS)
        ls_epoch = []
        ls_lr = []
        for epoch in range(NUM_EPOCHS):
            running_loss = 0.0
            saving_loss = 0.0
            ls_epoch.append(epoch)
            ls_lr.append(optimizer.param_groups[0]["lr"])
            scheduler.step()
        plt.plot(ls_epoch, ls_lr, label='Curve 2')
        
        plt.title('LR Plot')
        plt.xlabel('Epoch')
        plt.ylabel('LR')
        plt.legend()
        plt.show()

In [5]:
if __name__ == '__main__':
    tester = Tester()
    tester.augment_tile = transforms.Compose([])
    [tester.process(load=True) for i in range(5)]
    
#     for i in range(8):
#         global model_path
#         model_path = f'./save/xxx{i+1}.pt'
#         tester.process()

Parameter: 87586674
Parameter: 87586674
Epoch: 100


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:36: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/782 [00:00<?, ?it/s]

[Epoch 0] Avg diff on the test set: 0.09
[Epoch 0] Accuracy on the test set: 96.37%
[Sample result]
tensor([[1., 0., 1., 0.],
        [1., 2., 1., 2.],
        [0., 0., 0., 0.],
        [1., 1., 1., 1.],
        [2., 2., 2., 2.],
        [0., 1., 0., 1.],
        [0., 2., 0., 2.],
        [2., 0., 2., 0.],
        [2., 1., 2., 1.]], device='cuda:3')
Accuracy: 100.00%
Parameter: 87586674
Parameter: 87586674
Epoch: 100


  0%|          | 0/782 [00:00<?, ?it/s]

...

[Epoch 0] Avg diff on the test set: 0.09
[Epoch 0] Accuracy on the test set: 96.43%
[Sample result]
tensor([[1., 1., 1., 1.],
        [0., 2., 0., 2.],
        [2., 0., 2., 0.],
        [0., 0., 0., 0.],
        [2., 1., 2., 1.],
        [2., 2., 2., 2.],
        [1., 0., 1., 0.],
        [1., 2., 1., 2.],
        [0., 1., 0., 1.]], device='cuda:3')
Accuracy: 100.00%
Parameter: 87586674
Parameter: 87586674
Epoch: 100


  0%|          | 0/782 [00:00<?, ?it/s]

[Epoch 0] Avg diff on the test set: 0.09
[Epoch 0] Accuracy on the test set: 96.35%
[Sample result]
tensor([[2., 2., 2., 2.],
        [1., 2., 1., 2.],
        [1., 1., 1., 1.],
        [2., 0., 2., 0.],
        [2., 1., 2., 1.],
        [0., 1., 0., 1.],
        [1., 0., 1., 0.],
        [0., 2., 0., 2.],
        [0., 0., 0., 0.]], device='cuda:3')
Accuracy: 100.00%
Parameter: 87586674
Parameter: 87586674
Epoch: 100


  0%|          | 0/782 [00:00<?, ?it/s]

[Epoch 0] Avg diff on the test set: 0.09
[Epoch 0] Accuracy on the test set: 96.44%
[Sample result]
tensor([[1., 1., 1., 1.],
        [1., 2., 1., 2.],
        [1., 0., 1., 0.],
        [0., 1., 0., 1.],
        [2., 1., 2., 1.],
        [2., 0., 2., 0.],
        [0., 0., 0., 0.],
        [2., 2., 2., 2.],
        [0., 2., 0., 2.]], device='cuda:3')
Accuracy: 100.00%


In [6]:
'''
31: 71.72%
32: 83.23%
35: 86.27%
36: 87.25%
39: 88.82%
40: 89.24%
53: 92.82%
57: 93.35%
62: 94.20%
71: 95.10%
83: 95.99%


5 test mean(CNN): 88.1%
(88.15 + 87.98 + 88.12 + 88.31 + 88.07)/5

5 test mean(ViT): 96.4%
(96.37 + 96.25 + 96.43 + 96.35 + 96.44)/5
'''

'\n31: 71.72%\n32: 83.23%\n35: 86.27%\n36: 87.25%\n39: 88.82%\n40: 89.24%\n53: 92.82%\n57: 93.35%\n62: 94.20%\n71: 95.10%\n83: 95.99%\n\n\n5 test mean(CNN): 88.1%\n(88.15 + 87.98 + 88.12 + 88.31 + 88.07)/5\n\n5 test mean(ViT): \n'